xAI Grok 4 is released on 2025-07-10, let us try it out

In [1]:
from openai import OpenAI
import os 

In [2]:
client = OpenAI(
  api_key=os.getenv("XAI_API_KEY"),
  base_url="https://api.x.ai/v1",
)

In [3]:
def ask_grok(prompt, client=None, model="grok-4", base_url="https://api.x.ai/v1"):
    if client is None:
        client = OpenAI(
            api_key=os.getenv("XAI_API_KEY"),
            base_url=base_url,
        )
        
    response = client.chat.completions.create(
        model=model,  # "grok-3",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    
    return parse_grok_response(response=response)

In [4]:
# Grok 4 Response Parser and Usage Examples

def parse_grok_response(response):
    """
    Parse Grok 4 ChatCompletion response to extract key information
    
    Args:
        response: ChatCompletion object from Grok 4 API
        
    Returns:
        dict: Parsed response data
    """
    try:
        # Extract message content
        message_content = response.choices[0].message.content
        
        # Extract model name
        model = response.model
        
        # Extract total tokens
        total_tokens = response.usage.total_tokens
        
        # Additional useful information
        parsed_data = {
            'message_content': message_content,
            'model': model,
            'total_tokens': total_tokens,
            'prompt_tokens': response.usage.prompt_tokens,
            'completion_tokens': response.usage.completion_tokens,
            'reasoning_tokens': response.usage.completion_tokens_details.get('reasoning_tokens', 0),
            'finish_reason': response.choices[0].finish_reason,
            'response_id': response.id,
            'created': response.created
        }
        
        return parsed_data
        
    except (AttributeError, IndexError, KeyError) as e:
        print(f"Error parsing response: {e}")
        return None

In [7]:
prompt = "5 的阶乘是多少？"

In [8]:
completion = client.chat.completions.create(
  model="grok-4",  # "grok-3",
  messages=[
    {"role": "user", "content": prompt}
  ]
)

In [ ]:
resp = parse_grok_response(response=completion)

In [10]:
resp.update({"prompt": prompt})

In [11]:
resp 

{'message_content': '5 的阶乘（5!）是 5 × 4 × 3 × 2 × 1 = **120**。  \n\n如果你有其他数字的阶乘或其他问题，随时问我！😊',
 'model': 'grok-4-0709',
 'total_tokens': 163,
 'prompt_tokens': 15,
 'completion_tokens': 45,
 'reasoning_tokens': 103,
 'finish_reason': 'stop',
 'response_id': 'db1a4cf8-b289-31a1-ed34-e65f2c4c4097',
 'created': 1752148189,
 'prompt': '5 的阶乘是多少？'}

In [14]:
prompt = "用Python写一个函数来计算斐波那契数列的第n项"
resp = ask_grok(prompt, client=client)

In [16]:
resp

{'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
 'model': 'grok-4-0709',
 'total_tokens': 697,
 'prompt_tokens': 24,
 'completion_tokens': 301,
 'reasoning_tokens': 372,
 'finish_reason': 'stop',
 'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
 'created': 1752

In [5]:
import re
from IPython.display import display, Markdown, HTML
try:
    from IPython.display import Code
except ImportError:
    # Fallback for older IPython versions
    Code = None

def display_grok_response(parsed_response, show_stats=True):
    """
    Display Grok response content nicely in Jupyter Notebook
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        show_stats (bool): Whether to show token usage statistics
    """
    
    content = parsed_response.get('message_content', '')
    
    if show_stats:
        # Display token usage statistics
        stats_html = f"""
        <div style="background-color: #f0f8ff; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
            <h4>📊 Token Usage Statistics</h4>
            <ul>
                <li><strong>Model:</strong> {parsed_response.get('model', 'N/A')}</li>
                <li><strong>Total Tokens:</strong> {parsed_response.get('total_tokens', 0)}</li>
                <li><strong>Prompt Tokens:</strong> {parsed_response.get('prompt_tokens', 0)}</li>
                <li><strong>Completion Tokens:</strong> {parsed_response.get('completion_tokens', 0)}</li>
                <li><strong>Reasoning Tokens:</strong> {parsed_response.get('reasoning_tokens', 0)}</li>
                <li><strong>Finish Reason:</strong> {parsed_response.get('finish_reason', 'N/A')}</li>
            </ul>
        </div>
        """
        display(HTML(stats_html))
    
    # Display the main content
    display(Markdown("### 🤖 Grok Response:"))
    display(Markdown(content))

def extract_code_blocks(content):
    """
    Extract code blocks from markdown content
    
    Args:
        content (str): Content with potential code blocks
        
    Returns:
        list: List of dictionaries with 'language' and 'code' keys
    """
    # Pattern to match code blocks with language specification
    pattern = r'```(\w+)?\n(.*?)```'
    matches = re.findall(pattern, content, re.DOTALL)
    
    code_blocks = []
    for match in matches:
        language = match[0] if match[0] else 'text'
        code = match[1].strip()
        code_blocks.append({
            'language': language,
            'code': code
        })
    
    return code_blocks

def display_code_blocks(parsed_response):
    """
    Extract and display code blocks separately for easy copying
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found in the response.")
        return
    
    for i, block in enumerate(code_blocks, 1):
        display(HTML(f"<h4>📝 Code Block {i} ({block['language']})</h4>"))
        
        # Use Code if available, otherwise fall back to HTML with syntax highlighting
        if Code:
            display(Code(block['code'], language=block['language']))
        else:
            # Fallback: display as preformatted text with basic styling
            code_html = f"""
            <div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; 
                        border-left: 4px solid #007acc; margin: 10px 0;">
                <pre style="margin: 0; overflow-x: auto;"><code>{block['code']}</code></pre>
            </div>
            """
            display(HTML(code_html))

def extract_and_save_code(parsed_response, filename=None):
    """
    Extract code blocks and optionally save to file
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        filename (str): Optional filename to save the code
        
    Returns:
        list: List of code blocks
    """
    content = parsed_response.get('message_content', '')
    code_blocks = extract_code_blocks(content)
    
    if not code_blocks:
        print("No code blocks found.")
        return []
    
    # If filename provided, save the first code block
    if filename and code_blocks:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(code_blocks[0]['code'])
        print(f"Code saved to {filename}")
    
    return code_blocks

def quick_test_code(parsed_response):
    """
    Quick test function - extracts first Python code block and returns it for execution
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        
    Returns:
        str: Python code ready for execution
    """
    code_blocks = extract_code_blocks(parsed_response.get('message_content', ''))
    
    python_blocks = [block for block in code_blocks if block['language'].lower() == 'python']
    
    if not python_blocks:
        print("No Python code blocks found.")
        return ""
    
    return python_blocks[0]['code']

# Enhanced display function with everything
def display_grok_analysis(parsed_response, extract_code=True, show_stats=True):
    """
    Complete analysis and display of Grok response
    
    Args:
        parsed_response (dict): Parsed response from parse_grok_response()
        extract_code (bool): Whether to extract and display code blocks separately
        show_stats (bool): Whether to show token usage statistics
    """
    
    # Show statistics
    if show_stats:
        display_grok_response(parsed_response, show_stats=True)
    else:
        display_grok_response(parsed_response, show_stats=False)
    
    # Extract and display code blocks
    if extract_code:
        print("\n" + "="*50)
        display(HTML("<h3>🔧 Code Extraction</h3>"))
        display_code_blocks(parsed_response)

# Example usage functions
def fibonacci_test_example():
    """Example of how to use the helper with your Fibonacci response"""
    
    # Your actual Fibonacci response
    fibonacci_response = {
        'message_content': '```python\ndef fibonacci(n):\n    """\n    计算斐波那契数列的第n项（从0开始索引）。\n    例如：\n    - fibonacci(0) = 0\n    - fibonacci(1) = 1\n    - fibonacci(2) = 1\n    - fibonacci(3) = 2\n    - fibonacci(4) = 3\n    - fibonacci(5) = 5\n\n    参数:\n    n (int): 非负整数，表示斐波那契数列的索引。\n\n    返回:\n    int: 斐波那契数列的第n项。\n\n    异常:\n    ValueError: 如果n为负数。\n    """\n    if n < 0:\n        raise ValueError("n must be a non-negative integer")\n    if n == 0:\n        return 0\n    if n == 1:\n        return 1\n    \n    a, b = 0, 1\n    for _ in range(2, n + 1):\n        a, b = b, a + b\n    return b\n\n# 示例用法\nif __name__ == "__main__":\n    print(fibonacci(0))  # 输出: 0\n    print(fibonacci(1))  # 输出: 1\n    print(fibonacci(5))  # 输出: 5\n    print(fibonacci(10)) # 输出: 55\n```',
        'model': 'grok-4-0709',
        'total_tokens': 697,
        'prompt_tokens': 24,
        'completion_tokens': 301,
        'reasoning_tokens': 372,
        'finish_reason': 'stop',
        'response_id': 'bb50a4de-ca80-0d5e-ab8f-ce5b4ddfb758',
        'created': 1752149057
    }
    
    # Display with full analysis
    display_grok_analysis(fibonacci_response)
    
    # Extract code for testing
    code = quick_test_code(fibonacci_response)
    print("\n" + "="*50)
    print("🚀 Ready to test! Use this code:")
    print("="*50)
    return code

# Simple usage examples
"""
# Basic usage:
display_grok_response(your_parsed_response)

# With code extraction:
display_grok_analysis(your_parsed_response)

# Just extract code for testing:
code = quick_test_code(your_parsed_response)
exec(code)  # Execute the code

# Save code to file:
extract_and_save_code(your_parsed_response, "fibonacci.py")
"""

'\n# Basic usage:\ndisplay_grok_response(your_parsed_response)\n\n# With code extraction:\ndisplay_grok_analysis(your_parsed_response)\n\n# Just extract code for testing:\ncode = quick_test_code(your_parsed_response)\nexec(code)  # Execute the code\n\n# Save code to file:\nextract_and_save_code(your_parsed_response, "fibonacci.py")\n'

In [19]:
display_grok_response(resp)

### 🤖 Grok Response:

```python
def fibonacci(n):
    """
    计算斐波那契数列的第n项（从0开始索引）。
    例如：
    - fibonacci(0) = 0
    - fibonacci(1) = 1
    - fibonacci(2) = 1
    - fibonacci(3) = 2
    - fibonacci(4) = 3
    - fibonacci(5) = 5

    参数:
    n (int): 非负整数，表示斐波那契数列的索引。

    返回:
    int: 斐波那契数列的第n项。

    异常:
    ValueError: 如果n为负数。
    """
    if n < 0:
        raise ValueError("n must be a non-negative integer")
    if n == 0:
        return 0
    if n == 1:
        return 1
    
    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

# 示例用法
if __name__ == "__main__":
    print(fibonacci(0))  # 输出: 0
    print(fibonacci(1))  # 输出: 1
    print(fibonacci(5))  # 输出: 5
    print(fibonacci(10)) # 输出: 55
```

In [6]:
def fibonacci(n):
    """
    计算斐波那契数列的第n项（从0开始索引）。
    例如：
    - fibonacci(0) = 0
    - fibonacci(1) = 1
    - fibonacci(2) = 1
    - fibonacci(3) = 2
    - fibonacci(4) = 3
    - fibonacci(5) = 5

    参数:
    n (int): 非负整数，表示斐波那契数列的索引。

    返回:
    int: 斐波那契数列的第n项。

    异常:
    ValueError: 如果n为负数。
    """
    if n < 0:
        raise ValueError("n must be a non-negative integer")
    if n == 0:
        return 0
    if n == 1:
        return 1

    a, b = 0, 1
    for _ in range(2, n + 1):
        a, b = b, a + b
    return b

# 示例用法
if __name__ == "__main__":
    print(fibonacci(0))  # 输出: 0
    print(fibonacci(1))  # 输出: 1
    print(fibonacci(5))  # 输出: 5
    print(fibonacci(10)) # 输出: 55

0
1
5
55


In [7]:
# prompt = "When was the largest firework show in Japan on July 4th"

prompt = "What is square root of 2"

resp = ask_grok(prompt, client=client)
display_grok_response(resp)

### 🤖 Grok Response:

Based on your query, I'll assume you're asking for the mathematical value of the square root of 2 (often written as √2). Let me break it down step by step, including the exact value, an approximation, and some context!

### 1. **Exact Value**
- The square root of 2 is the number that, when multiplied by itself, equals 2. In other words, if \( x = \sqrt{2} \), then \( x \times x = 2 \).
- It's an **irrational number**, meaning it can't be expressed as a simple fraction (like 1/2 or 3/4) and its decimal goes on forever without repeating.
- Exact mathematical notation: \( \sqrt{2} \) or \( 2^{1/2} \).

### 2. **Approximate Value**
- A common decimal approximation is **1.41421356237** (rounded to 11 decimal places).
- For everyday use:
  - To 2 decimals: ≈ 1.41
  - To 3 decimals: ≈ 1.414
  - To 5 decimals: ≈ 1.41421
- You can get more precise values using a calculator or tools like Python/Google/Wolfram Alpha.

### 3. **How to Calculate It**
- **Manually**: You can use methods like the Babylonian algorithm (a type of iterative approximation). Start with a guess (e.g., 1.4), square it, and refine:
  - Guess: 1.4 → 1.4² = 1.96 (too low)
  - Refine: Average of 1.4 and (2/1.4) ≈ 1.414 → 1.414² ≈ 1.999 (very close!)
  - Repeat for more precision.
- **Using a Calculator**: Just type √2 or "sqrt(2)" into most calculators or search engines.
- **In Programming**: In Python, it's `math.sqrt(2)` which gives ~1.4142135623730951.

### 4. **Fun Facts and Context**
- √2 is famous in math because it's the length of the diagonal of a square with sides of length 1 (from the Pythagorean theorem: diagonal = √(1² + 1²) = √2).
- It was proven irrational by ancient Greek mathematicians (like Hippasus) around 500 BC, which was a big deal at the time!
- It appears in nature, geometry, and physics (e.g., in wave patterns or the golden ratio approximations).

If this isn't what you meant (e.g., if you wanted the square root of a different number, or something like "square root of 2" in a non-math context like a riddle or code), feel free to provide more details! 😊